In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/requirements.txt'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
import skfuzzy as fuzz
import skfuzzy.control as ctrl

In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/utils.py .
from utils import preprocess_tweet

# Análisis inteligente de datos guiado por ingeniería del conocimiento, un caso práctico: radicalización y odio en redes sociales.

A lo largo del taller se mostrarán ejemplos explícitos de Discurso de Odio para ilustrar el desarrollo. La finalidad del taller es formativa y en ningún momento se pretende ofender al público. Para paliar la lacra del contenido de odio en la red es necesario conocerlo.




El **Departamento de Seguridad Nacional** nos ha encargado un prototipo de un **sistema de alerta temprana** que les avise si se produce algún altercado de odio en Twitter. 

El sistema deberá **detectar el discurso de odio en español en la red social Twitter** y **avisar** al Departamente de Seguridad Nacional **únicamente en el caso de que el discurso de odio sea de tal calado que pueda estar cerca de suponer un peligro para la convivencia pacífica y el orden público**.

Recopilando, necesitamos:



1.   **Un sistema que automáticamente nos diga si un mensaje es de odio o no es de odio.**
2.   **Un modelo de razonamiento que nos permita inferir la intensidad del discurso de odio en base a una serie de parámetros predefinidos.**



## Clasificación automáticas de mensajes de odio.

Para resolver el problema de clasificación automática de mensajes de odio vamos a emplear el conjunto de datos **HATERNET**. Este conjunto de datos está formado por 6000 tweets etiquetados manualmente como odio y no odio.

*Pereira-Kohatsu, J.C.; Quijano-Sánchez, L.; Liberatore, F.; Camacho-Collados, M. Detecting and Monitoring Hate Speech in Twitter. Sensors 2019, 19, 4654. https://doi.org/10.3390/s19214654*

### Análisis exploratorio de datos

Análisis preliminar de un conjunto de datos para resumir sus principales características, donde a menudo se emplean métodos visuales.

In [ ]:
# Leemos las 10 primeras líneas
haternet_path = "/content/drive/MyDrive/Colab Notebooks/HATERNET.txt"
with open(haternet_path) as archivo:
    head = [next(archivo) for x in range(10)]
print("\n".join(head))

In [ ]:
# Cargamos el dataset
haternet_df = pd.read_csv(haternet_path, sep=..., names=['id','tweet','clase'], engine='python')

In [ ]:
# Obtenemos información del data frame y los tipos de variable
haternet_df.info()

In [ ]:
# Miramos la distribución de clases
print(...(haternet_df['...']))

In [ ]:
# Cambiamos los tipos de datos
haternet_df = haternet_df.astype({"id": "string", "tweet": "string", "clase":"category"})

In [ ]:
# Vamos a ver algunas estadísticas
print(haternet_df.describe())

In [ ]:
# Miramos el tweet duplicado
haternet_df[haternet_df['tweet'].isin(haternet_df['tweet'][haternet_df['tweet'].duplicated()])]

In [ ]:
# Vemos la distribución de las clases
sns.catplot(data=haternet_df, x="clase", kind="count", palette="ch:.25")

In [ ]:
# Hacemos una copia del dataframe
haternet_df_copia = haternet_df
# Creamos una nueva columna con la longitud de los mensajes
haternet_df_copia['longitud'] = ...

In [ ]:
# Vamos a ver la distribución de longitud de los mensajes
sns.displot(..., x="...",bins=20)

In [ ]:
# Primero aquellos tweets de más de 280 caracteres
pd.options.display.max_colwidth = 400
print(haternet_df_copia["tweet"][haternet_df_copia.longitud > 280])

In [ ]:
# Vamos a seleccionar uno aleatorio
haternet_df_copia["tweet"].loc[np.random.choice(haternet_df_copia["tweet"][...].index)]

In [ ]:
# Miramos los tweets cortos
print(haternet_df_copia["tweet"][...])

In [ ]:
# Unimos los mensajes
texto_odio = (" ").join(haternet_df["tweet"][haternet_df.clase == 1])
texto_noOdio = ...

In [ ]:
# Generamos una nube de términos para los tweets etiquetados como odio
wordcloud_odio = WordCloud().generate(texto_odio)
plt.imshow(wordcloud_odio,interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Generamos una nube de términos para los tweets etiquetados como no odio
wordcloud_noOdio = WordCloud().generate(texto_noOdio)
plt.imshow(wordcloud_noOdio,interpolation='bilinear')
plt.axis("off")
plt.show()

### Preprocesamiento

En esta fase se manipulan los mensajes para dejar aquel contenido útil para la posterior fase de análisis. En la fase de preprocesamiento incluye la limpieza, eliminación de ruido y, también una representación intermedia previa al proceso de transformación.

In [ ]:
# Eliminar el contenido duplicado
haternet_df = haternet_df.drop_duplicates(subset=['...'])

In [ ]:
# Comprobamos que ya no hay duplicados
print(...)

In [ ]:
# Balanceamos las clases
# Función random undersampling
def RUS(df):
    clase_positiva = df[df['clase']==1]
    clase_negativa = ...
    
    num_pos = len(clase_positiva)
    num_neg = ...

    if (num_pos > num_neg):
        fraccion = num_neg / num_pos
        submuestreo = pd.concat([clase_positiva.sample(frac = fraccion, random_state = 78),clase_negativa],axis=0)
    else:
        fraccion = ...
        submuestreo = ...
    

    return submuestreo.sort_index().reset_index()

In [ ]:
# Aplicamos la función
haternet_df_b = ...

In [ ]:
# Comprobamos que los tweets están balanceados
sns.catplot(data=..., x="...", kind="...", palette="ch:.25")

La **tokenización** es la división del texto en unidades individuales denominadas tokens que posteriormente actuarán de características para nuestro modelo.

In [ ]:
# Cargamos el tokenizador
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

In [ ]:
# Tokenizamos
haternet_df_b['tokens'] = [tokenizer.tokenize(tweet) for tweet in haternet_df_b['tweet']]

In [ ]:
# Mostramos el dataframe para ver la tokenización
haternet_df_b

### Empezamos el proceso de limpieza de los mensajes

In [ ]:
# Eliminamos los emojis
def eliminarEmojis(texto):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticonos
        u"\U0001F300-\U0001F5FF"  # símbolos
        u"\U0001F680-\U0001F6FF"  # transporte y mapas
        u"\U0001F1E0-\U0001F1FF"  # banderas
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',texto)

haternet_df_b['tokens'] = haternet_df_b['tokens'].apply(lambda tokens: [eliminarEmojis(palabra) for palabra in tokens])

In [ ]:
# Comprobamos que se han elimiado los emojis
haternet_df_b['tokens']

In [ ]:
# Eliminamos las urls
# http o https o www seguido de cualquier caracter
url_rx = re.compile(r'(http(s)?).+|www\..+')
haternet_df_b['tokens'] = haternet_df_b['tokens'].apply(lambda tokens: [palabra for palabra in tokens if not url_rx.match(palabra)])

In [ ]:
# Comprobamos si las URLs se han eliminado
haternet_df_b

In [ ]:
# Eliminamos los números
numeros_rx = re.compile(r'^[0-9]+$')
haternet_df_b['tokens'] = ...

In [ ]:
# Comprobamos que se han eliminado los números
haternet_df_b

In [ ]:
# Eliminamos los signos de puntuación
haternet_df_b['tokens'] = haternet_df_b['tokens'].apply(lambda tokens: [re.sub(r'^([^\w]|_)+$', '', palabra) for palabra in tokens])

In [ ]:
# Comprobamos que se han eliminado los signos de puntuación
...

In [ ]:
# Eliminamos las palabras vacías
# Observamos la lista de palabras vacías que nos proporciona la librería nltk
print(stopwords.words('spanish'))
# Vemos que las palabras está acentuadas y como bien sabemos Twitter no se define precisamente por sus buenos escritores

In [ ]:
# Replicamos la lista de palabras vacías con los acentos eliminados.
def eliminar_acentos(texto):
    texto = re.sub(r"[àáâãäå]", 'a', texto)
    texto = re.sub(r"[èéêë]", 'e', texto)
    texto = re.sub(r"[ìíîï]", 'i', texto)
    texto = re.sub(r"[òóôõö]", 'o', texto)
    texto = re.sub(r"[ùúûü]", 'u', texto)
    return texto

stop_words_sin_acentuar = ...


In [ ]:
# Miramos la lista de las palabras vacías sin acentuar
print(...)

In [ ]:
# Eliminamos las palabras vacías
haternet_df_b['tokens'] = haternet_df_b['tokens'].apply(lambda tokens: [palabra for palabra in tokens if palabra not in stopwords.words('spanish') and ...])

In [ ]:
# Comprobamos que se han eliminado las palabras vacías
...

In [ ]:
# Definimos algunos términos de propios de la jerga de twitter y los eliminamos
# Por ejemplo el jajaja y el xd
jerga_rx = re.compile(r'^(j(j)?(a)?)[aj]*|(xd)[d]*$')
# Eliminamos esos tokens
haternet_df_b['tokens'] = ...

In [ ]:
# Vamos a eliminar los huecos vacíos que han quedado después de limpiar los mensajes
# Eliminamos los tokens vacíos
haternet_df_b['tokens'] = haternet_df_b['tokens'].apply(lambda tokens: [palabra for palabra in tokens if ...])

**Segmentación (Stemming) y lematización** son métodos empleados para **reducir el tamaño del vocabulario** buscando su forma morfológica, raíz o lemas.



In [ ]:
# Stemming de los mensajes
stemmer_esp = SnowballStemmer('spanish')

haternet_df_b['tokens'] = haternet_df_b['tokens'].apply(...)

In [ ]:
# Volvemos a visualizar la nube de términos
# Unimos los tokens en un solo string
texto = (" ").join(token for tweet  in haternet_df_b["tokens"] for token in tweet)
# Visualizamos la frecuencia de las palabras
wordcloud_haternet = WordCloud().generate_from_text(texto)
plt.imshow(wordcloud_haternet,interpolation='bilinear')
plt.axis("off")
plt.show()

### Transformación

Fase en la que **transformamos nuestros mensajes para que puedan servir como entrada a un algoritmo de aprendizaje**. Esta transformación puede llevarse a cabo mediante la aplicación de diversos métodos como la **vectorización con bolsas de palabras** o los **word embedings**.

In [ ]:
# Vectorización 
# Definimos la función TF-IDF
vectorizer = TfidfVectorizer(lowercase=False, min_df = 15)

In [ ]:
# Creamos un corpus con todos los mensajes del dataset eliminando la tokenización
corpus = [TreebankWordDetokenizer().detokenize(tweet) for tweet in haternet_df_b['tokens']]

In [ ]:
# Vectorizamos la entrada
X = vectorizer.fit_transform(corpus)
# Asiganmos las etiquetas a una nueva variable
y = haternet_df_b['clase']

In [ ]:
# Mostramos el número de documentos y el de características
print(X.shape)

In [ ]:
# Observamos algunos de nuestros términos que actúan como características
print(vectorizer.get_feature_names_out())

In [ ]:
# Vemos la estructura de nuestros parámetros de entrada
print(X.toarray()[0])

### Clasificación

Para la clasificicación de mensajes de odio vamos a emplear el algoritmo llamado **Máquinas de Soporte Vectorial (SVM)**. El motivo es porque en la literatura es el más empleado para esta tarea.

In [ ]:
# Dividimos el conjunto de datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Cargamos el algoritmo
svc = svm.SVC()

In [ ]:
# Definimos la búsqueda en grid asignándole varios valores a cada uno de los hiperparámetros
param_grid = {'C': [0.1, 1, 10, 50, 100, 1000],  
              'gamma': [0.01, 0.001, 0.0001, 0.00001],
              'kernel': ['rbf']}

In [ ]:
# El verbose es para que nos muestre información
# Definimos el modelo
model = GridSearchCV(estimator = ..., param_grid = ..., cv = 5, verbose = 3)

In [ ]:
# Entrenamos
model.fit(X = ..., y = ...)

### Evaluación

Para la evaluación del modelo vamos a emplear una serie de métricas que nos van a dar una visión sobre cómo ha funcionado nuestro modelo frente a ejemplos no vistos en el proceso de entrenamiento.





In [ ]:
# Seleccionamos el mejor modelo
mejor_modelo = model.best_estimator_

In [ ]:
# Lo probamos con nuestro test
y_pred = ...

In [ ]:
# Observamos la matriz de confusión
plot_confusion_matrix(mejor_modelo, X_test, y_test)  
plt.show()

In [ ]:
# Mostramos las métricas 
print(classification_report(..., ...))

## Motor de inferencia para evaluar la intensidad del discurso de odio.

Para medir la intensidad del discurso de odio vamos a emplear un mecanismo de inferencia **basado en palabras** en lugar de en números. Esto se conoce como **computación con palabras** o *computing with words* en inglés. El término lo acuñó el profesor Lotfi Zadeh, creador de la lógica borrosa.

Para construir el motor de razonamiento necesitamos una base de conocimiento.



Una base de conocimiento está formada por:

*   Base de datos que contiene los conjuntos de términos lingüísticos empleados para describir cada una de las clases de la taxonomía y funciones de pertenencia asociadas a estos términos lingüísticos que definen la semántica dentro de un dominio predefinido. Esas funciones de pertenencia pueden ser triangulares o trapezoidales entre otras.
*   Base de reglas que está formada por una colección de reglas lingüísticas con la siguiente estructura: 
$$\text{Si  } X_1 \text{ es } A_1 \text{ y } \text{ ... } \text{ y } X_n \text{ es } A_n \text{ entonces } Y \text{ es } B$$



**Definición del universo de discurso del primer nivel de la taxonomía**


In [ ]:
# Variables borrosas
seguidores = ctrl.Antecedent(np.arange(0,90001,1), 'seguidores')
# Universo de discurso entre 0 y 1 con una granularidad de 0.01
tipo_emisor = ...

influencia_emisor_cons = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'influencia_emisor')

# Universo de discurso entre 0 y 20000 con una granularidad de 1
likes = ...
retweets = ctrl.Antecedent(np.arange(0,10001,1), 'retweets')

# Universo de discurso entre 0 y 1 con una granularidad de 0.01
impacto_mensaje_cons = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'impacto_mensaje')


**Definición de las funciones de pertenencia del primer nivel de la taxonomía**

In [ ]:
# Número de seguidores
seguidores['Pocos'] = fuzz.trimf(seguidores.universe, [0, 0, 5000])
seguidores['Algunos'] = fuzz.trapmf(seguidores.universe, [0, 5000, 10000,20000])
seguidores['Bastantes'] = fuzz.trapmf(seguidores.universe, [10000, 20000, 40000, 60000])
seguidores['Muchos'] = fuzz.trimf(seguidores.universe, [40000, 90000, 90000])

In [ ]:
# Vamos a ver algunas de las funciones de pertenencia definidas en base a un conocimiento experto
seguidores.view()

In [ ]:
# Tipo de emisor
etiquetas_tip_emi = ['No_verificada','Verificada']
tipo_emisor['No_verificada'] = fuzz.trimf(tipo_emisor.universe, [0, 0, 1])
tipo_emisor['Verificada'] = fuzz.trimf(tipo_emisor.universe, [0, 1, 1])

# Influencia del emisor
etiquetas_inf_em = ['Muy_baja','Baja','Media','Alta','Muy_alta']
influencia_emisor_cons.automf(names = etiquetas_inf_em)
# Likes
# Etiquetas lingüísticas Pocos, Bastantes y Muchos con funciones de pertenencia triangular, trapezoidal y triangular respectivamente
# La primera función de pertenencia triangular va de 0 a 1.000 con el mayor grado de pertenencia en el 0
likes...
# La segunda función de pertenencia trapezoidal va del 0 a 10.000 con el máximo grado de pertenencia entre el 1.000 y el 2.000
likes...
# La tercera función de pertenencia triangular va del 2.000 al 20.000 con el mayor grado de pertenencia en el 20.000
likes...

# Retweets
retweets['Pocos'] = fuzz.trimf(retweets.universe, [0, 0, 500])
retweets['Bastantes'] = fuzz.trapmf(retweets.universe, [0, 800, 1500, 5000])
retweets['Muchos'] = fuzz.trimf(retweets.universe, [1500, 10000, 10000])

# Impacto del mensaje en la sociedad
# Todas las funciones de pertenencia son iguales y las etiquetas son: Bajo, Medio, Alto
etiquetas_imp_men = ...
impacto_mensaje_cons ...

In [ ]:
# Observamos la función de pertenencia del tipo de emisor
...

In [ ]:
# Observamos la función de pertenencia de la influencia del emisor
...

In [ ]:
# Observamos la función de pertenencia de los likes
likes.view()

In [ ]:
# Observamos la función de pertenencia de los retweets
retweets.view()

In [ ]:
# Observamos la función de pertenencia del impacto del mensaje
impacto_mensaje_cons.view()

**Definición del universo de discurso del segundo nivel de la taxonomía**

In [ ]:
# En todas el universo de discurso es entre 0 y 1
# Influencia del emisor
influencia_emisor_ant = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'influencia_emisor')
# Impacto del mensaje
impacto_mensaje_ant = ...
# Intensidad del discurso de odio
intensidad_odio = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'intensidad_odio')

**Definición de las funciones de pertenencia del segundo nivel de la taxonomía**

In [ ]:
# Todas las funciones de pertenencia son iguales
# Influencia del emisor
influencia_emisor_ant ...
# Impacto del mensaje en la sociedad
impacto_mensaje_ant ...
# Intensidad del discurso de odio
etiquetas_int_odio = ['Muy_leve','Leve','Medio','Grave','Muy_grave']
intensidad_odio ...

In [ ]:
# Visualizamos la función de pertenencia de la intensidad del discurso de odio
intensidad_odio.view()

**Definición de la base de reglas que relaciona las clases de la taxonomía**

In [ ]:
# Definición de las reglas para obtener la influencia del emisor
regla_ie_1 = ctrl.Rule(antecedent=(seguidores['Pocos'] & tipo_emisor['No_verificada']), consequent = (influencia_emisor_cons['Muy_baja'])
# Si el número de seguidores es Algunos y el tipo de emisor es de una cuenta Verificada entonces la ingluencia del emisor es Alta
regla_ie_2 = ...
regla_ie_3 = ctrl.Rule(antecedent=(seguidores['Bastantes'] & tipo_emisor['No_verificada']), consequent = (influencia_emisor_cons['Baja']))
regla_ie_4 = ctrl.Rule(antecedent=(seguidores['Muchos'] & tipo_emisor['No_verificada']), consequent = (influencia_emisor_cons['Media']))
regla_ie_5 = ctrl.Rule(antecedent=(seguidores['Bastantes'] & tipo_emisor['Verificada']), consequent = (influencia_emisor_cons['Muy_alta']))

In [ ]:
# Definición de las reglas para obtener el impacto del mensaje
regla_im_1 = ctrl.Rule(antecedent=(likes['Pocos'] & retweets['Pocos']), consequent=(impacto_mensaje_cons['Bajo']))
regla_im_2 = ctrl.Rule(antecedent=(likes['Bastantes'] & retweets['Pocos']), consequent=(impacto_mensaje_cons['Bajo']))
# Si los likes son Pocos y los retweets Bastantes entonces el impacto del mensaje es Medio
regla_im_3 = ...
regla_im_4 = ctrl.Rule(antecedent=(likes['Bastantes'] & retweets['Bastantes']), consequent=(impacto_mensaje_cons['Alto']))
regla_im_5 = ctrl.Rule(antecedent=(likes['Muchos'] & retweets['Muchos']), consequent=(impacto_mensaje_cons['Alto']))

In [ ]:
# Definición de las reglas para obtener la intensidad del discurso de odio
regla_io_1 = ctrl.Rule(antecedent=(influencia_emisor_ant['Muy_baja'] & impacto_mensaje_ant['Bajo']), consequent=(intensidad_odio['Muy_leve']))
regla_io_2 = ctrl.Rule(antecedent=(influencia_emisor_ant['Baja'] & impacto_mensaje_ant['Bajo']), consequent=(intensidad_odio['Leve']))
regla_io_3 = ctrl.Rule(antecedent=(influencia_emisor_ant['Media'] & impacto_mensaje_ant['Medio']), consequent=(intensidad_odio['Medio']))
# Si la influencia del emisor es Alta y el impacto del mensaje es Medio entonces la intensidad del discurso de odio es Grave
regla_io_4 = ...
regla_io_5 = ctrl.Rule(antecedent=(influencia_emisor_ant['Muy_alta'] & impacto_mensaje_ant['Alto']), consequent=(intensidad_odio['Muy_grave']))

Un motor de inferencia se describe como el proces de razonamiento que combina hechos y conocimiento para emitir una conclusión.




In [ ]:
# Motor de inferencia influencia del emisor
ctrl_ie = ctrl.ControlSystem(rules =[regla_ie_1, regla_ie_2,regla_ie_3,regla_ie_4,regla_ie_5])
motor_ie = ctrl.ControlSystemSimulation(ctrl_ie)

In [ ]:
# Motor de inferencia impacto del mensaje
ctrl_im = ctrl.ControlSystem(rules =[regla_im_1, regla_im_2,regla_im_3,regla_im_4,regla_im_5])
motor_im = ctrl.ControlSystemSimulation(ctrl_im)

In [ ]:
# Motor de inferencia intensidad del odio
ctrl_io = ...
motor_io = ...

### Ejemplo de ejecución del sistema completo

In [ ]:
# Vamos a plantear un ejemplo
df_example = pd.DataFrame(columns=["tweet", "seguidores", "tipo_emisor","likes","retweets","clase"], 
    data=[["putos moros de mierda que nos roban el trabajo",235,0,30,10,np.NaN],
        ["Si no hay trabajo para los españoles, menos para los inmigrantes. Fuera moros de nuestras fronteras", 15300, 1, 3500, 1200,np.NaN]])

In [ ]:
# Preprocesamos los mensajes
# Seleccionamos los tweets
X_prueba = ...
# Preprocesamos los tweets empleando la función preprocess_tweet
X_prueba = ...

In [ ]:
# Ayuda para conocer los parámetros de a funcion preprocess_tweet
help(preprocess_tweet)

In [ ]:
# Detokenizamos y añadimos esos nuevos mensajes a nuestro corpus
n_corpus = [TreebankWordDetokenizer().detokenize(tweet) for tweet in X_prueba]
corpus_actualizado = corpus + n_corpus

In [ ]:
# Volvemos a vectorizar
X_actualizada = ...

In [ ]:
# Seleccionamos únicamente los dos últimos mensajes
mensajes_vectorizados = X_actualizada.toarray()
prueba = mensajes_vectorizados[-2:]

In [ ]:
# Nueva prediccion
y_pred_nueva = ...

In [ ]:
# Vemos las etiquetas
y_pred_nueva

In [ ]:
# Las añadimos a nuestro ejemplo
df_example['clase'] = y_pred_nueva

In [ ]:
# Vamos a proporcinarle la entrada del primer mensaje a nuestro primer motor de inferencia para medir la influencia del emisor
motor_ie.input['seguidores'] = df_example.loc[0]['seguidores']
motor_ie.input['tipo_emisor'] = df_example.loc[0]['tipo_emisor']

In [ ]:
# Computamos
motor_ie.compute()

In [ ]:
# Observamos el resultado
print(motor_ie.output['influencia_emisor'])
influencia_emisor_cons.view(sim=motor_ie)

In [ ]:
# Función para observar qué reglas se han disparado
def rules_matching(ctrl_sys_sim):
  #Crear una lista para almacenar los resultados de las reglas
  rules_details = []
  #Recibe un simulador de sistema de control
  rules_count = 0
  #Lee las reglas asociadas
  for rule in ctrl_sys_sim.ctrl.rules.all_rules:
    rules_count+=1
    #Por cada regla, pasar el simulador de sistema de control
    rule_membership = rule.aggregate_firing[ctrl_sys_sim]
    if(rule_membership != 0):
      rules_details.append({'Regla':'Regla '+str(rules_count),
                            'Antecedentes':rule.antecedent,
                            'Consecuente':rule.consequent,
                            'Pertenencia':round(rule_membership,2)})
  return rules_details

# Observamos qué relgas se han disparado
rules_matching(motor_ie)

In [ ]:
# Proporcionamos la entrada del primer mensaje a nuestro segundo motor de inferencia para medir la influencia del emisor
motor_im.input['likes'] = ...
motor_im.input['retweets'] = ...
# Computamos
motor_im.compute()
# Observamos los resultados
print(motor_im.output['impacto_mensaje'])
impacto_mensaje_cons.view(sim=motor_im)

In [ ]:
# Observamos qué relgas se han disparado
rules_matching(motor_im)

In [ ]:
# Empleamos la salida de los dos motores de inferencia anteriores como entrada para medir la intensidad del discurso de odio
motor_io.input['influencia_emisor'] = motor_ie.output['influencia_emisor']
motor_io.input['impacto_mensaje'] = motor_im.output['impacto_mensaje']
# Computamos
motor_io.compute()
# Observamos los resultados
print(motor_io.output['intensidad_odio'])
intensidad_odio.view(sim=motor_io)

In [ ]:
# Observamos qué relgas se han disparado
...

In [ ]:
# Vamos con la segunda configuración del modelo, medimos la influencia del emisor
motor_ie.input['seguidores'] = ...
motor_ie.input['tipo_emisor'] = ...
# Computamos
...
# Observamos los resultados
print(motor_ie.output['influencia_emisor'])
influencia_emisor_cons.view(sim=motor_ie)

In [ ]:
# Observamos qué relgas se han disparado
rules_matching(motor_ie)

In [ ]:
# Medimos el impacto del mensaje
motor_im.input['likes'] = df_example.loc[1]['likes']
motor_im.input['retweets'] = df_example.loc[1]['retweets']
# Computamos
motor_im.compute()
# Observamos los resultados
print(motor_im.output['impacto_mensaje'])
impacto_mensaje_cons.view(sim=motor_im)

In [ ]:
# Observamos qué relgas se han disparado
rules_matching(motor_im)

In [ ]:
# Medimos la intensidad del discurso del odio
motor_io.input['influencia_emisor'] = motor_ie.output['influencia_emisor']
motor_io.input['impacto_mensaje'] = motor_im.output['impacto_mensaje']
# Computamos
motor_io.compute()
# Observamos los resultados
print(motor_io.output['intensidad_odio'])
intensidad_odio.view(sim=motor_io)

In [ ]:
# Observamos qué relgas se han disparado
rules_matching(motor_io)

# Gracias por vuestra atención

Profesor.AMontoro@uclm.es